# Feature Sub-Sampling 

In [1]:
from tqdm.auto import tqdm
from dh_grid import DHGrid
from sub_sampler import SubSampler
from matplotlib import cm

import matplotlib.pyplot as plt
import open3d as o3d
import numpy as np

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
export_ds = '/mnt/data/datasets/nuscenes/processed'
cloud_filename = f"{export_ds}/clouds.npy"
cloud_features = np.load(cloud_filename)

In [3]:
n_process = 10
cloud_features = cloud_features[0:n_process, :, :, :]
print(f"Shape of clouds is {cloud_features.shape}.")

Shape of clouds is (10, 3, 400, 400).


In [4]:
lf_bw = 100
sampler = SubSampler(cloud_features, lf_bw)
sampled_cloud_features = sampler.compute_output_data()
print(f"Shape of the subsampled clouds is {sampled_cloud_features.shape}.")


Shape of the subsampled clouds is (10, 3, 200, 200).


In [5]:
def mapIntensityToRGB(i):
    mask = np.where(i < 0)    
    colors = cm.jet(plt.Normalize(min(i), max(i))(i))
    colors[mask] = 0
    return colors

def prepare_for_viz(cloud):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(cloud[:, 0:3])
    if cloud.shape[1] == 4:
        colors = mapIntensityToRGB(cloud[:, 3])
        pcd.colors = o3d.utility.Vector3dVector(colors[:,0:3])
    if cloud.shape[1] == 6:
        pcd.colors = o3d.utility.Vector3dVector(cloud[:,3:6] / 255.0)
    return pcd

def visualize_pointcloud(cloud):
    pcd = prepare_for_viz(cloud)        
    o3d.visualization.draw_geometries([pcd])
    
def visualize_sphere(cloud, points):    
    cur_sem_cloud = np.reshape(cloud, (3, -1)).T
    points_xyz = DHGrid.ConvertPointsToEuclidean(points)        
    points_xyzi = np.column_stack((points_xyz, cur_sem_cloud[:,0]))
    visualize_pointcloud(points_xyzi)

In [8]:
hf_bw = int(cloud_features.shape[3] / 2)
_, hf_points = grid, points = DHGrid.CreateGrid(hf_bw)
visualize_sphere(cloud_features[0,:,:,:], hf_points)

In [7]:
_, lf_points = grid, points = DHGrid.CreateGrid(lf_bw)
visualize_sphere(sampled_cloud_features[0,:,:,:], lf_points)